# Create Combined All Data - Fixed Quarter Dates

1. credit_ratings_on_fixed_quarter_dates_with_earnings_call_date.csv  
        * Columns: rating, symbol, rating agency, rating_date, fixed_quarter_date, ...  
        * Size: 7981 × 16  
        * 587 unique companies
2. combined_calls.csv  
        * Columns: symbol, quarter', year, earnings_call_datetime, content, source, web, earnings_call_date  
        * Size: 31067 x 8  
        * 1646 unique companies  
3. combined_corrected_tabular_financial_statements_data.parquet  
        * Columns: date, symbol, reportedCurrency, period, filing_date, financial variables, Altman_Z ...  
        * Size: 20825 x 134  
        * 796 unique companies  

In [26]:
# Whether this is a sample/debug run or not
DEBUG = False

In [27]:
# Packages
import pandas as pd

In [28]:
# Load ~\Box\STAT 222 Capstone\Intermediate Data\Credit_Rating\credit_ratings_on_fixed_quarter_dates_with_earnings_call_date.csv
cr_and_dates = pd.read_csv(r'~\Box\STAT 222 Capstone\Intermediate Data\Credit_Rating\credit_ratings_on_fixed_quarter_dates_with_earnings_call_date.csv')
cr_and_dates

,ticker,fixed_quarter_date,earnings_call_date,Rating,Rating Agency Name,rating_date,Source,Rating Rank AAA is 10,Next Rating,Next Rating Date,Previous Rating,Previous Rating Date,next_rating_date_or_end_of_data,credit_rating_year,previous_fixed_quarter_date,source,days_since_call
0,AAPL,2014-07-01 00:00:00.000000,2014-04-23,AA,Standard & Poor's Ratings Services,2014-05-27,Supplementary,9,AA,2015-02-18,AAA,2014-04-24,2015-02-18,2014,2014-04-01 00:00:00.000000,web,69
1,AAPL,2014-10-01 00:00:00.000000,2014-07-22,AA,Standard & Poor's Ratings Services,2014-05-27,Supplementary,9,AA,2015-02-18,AAA,2014-04-24,2015-02-18,2014,2014-07-01 00:00:00.000000,web,71
2,AAPL,2015-01-01 00:00:00.000000,2014-10-20,AA,Standard & Poor's Ratings Services,2014-05-27,Supplementary,9,AA,2015-02-18,AAA,2014-04-24,2015-02-18,2014,2014-10-01 00:00:00.000000,web,73
3,AAPL,2015-04-01 00:00:00.000000,2015-01-27,AA,Standard & Poor's Ratings Services,2015-02-18,Supplementary,9,AA,2015-05-28,AA,2014-05-27,2015-05-28,2015,2015-01-01 00:00:00.000000,web,64
4,AAPL,2015-07-01 00:00:00.000000,2015-04-27,AA,Standard & Poor's Ratings Services,2015-06-02,Supplementary,9,AA,2015-08-25,AA,2015-05-28,2015-08-25,2015,2015-04-01 00:00:00.000000,web,65
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8002,ZTS,2015-10-01 00:00:00.000000,2015-08-04,BBB,Standard & Poor's Ratings Services,2015-01-30,Both,7,BBB,2015-11-03,BBB,2014-01-31,2015-11-03,2015,2015-07-01 00:00:00.000000,web,58
8003,ZTS,2016-01-01 00:00:00.000000,2015-11-03,BBB,Standard & Poor's Ratings Services,2015-11-03,Supplementary,7,BBB,2016-01-22,BBB,2015-01-30,2016-01-22,2015,2015-10-01 00:00:00.000000,web,59
8004,ZTS,2016-04-01 00:00:00.000000,2016-02-16,BBB,Standard & Poor's Ratings Services,2016-01-22,Both,7,BBB,2016-12-23,BBB,2015-11-03,2016-12-23,2016,2016-01-01 00:00:00.000000,web,45
8005,ZTS,2016-07-01 00:00:00.000000,2016-05-04,BBB,Standard & Poor's Ratings Services,2016-01-22,Both,7,BBB,2016-12-23,BBB,2015-11-03,2016-12-23,2016,2016-04-01 00:00:00.000000,web,58


In [29]:
# Convert earnings_call_date to date format
#cr_and_dates['earnings_call_date'] = pd.to_datetime(cr_and_dates['earnings_call_date']).dt.date

In [30]:
# drop source column (about to get it again via the next join)
cr_and_dates = cr_and_dates.drop(columns = ['source'])

In [31]:
# check for duplicates on ticker by fixed_quarter_date
print(cr_and_dates[cr_and_dates.duplicated(subset=['ticker', 'fixed_quarter_date'], keep=False)])

Empty DataFrame
Columns: [ticker, fixed_quarter_date, earnings_call_date, Rating, Rating Agency Name, rating_date, Source, Rating Rank AAA is 10, Next Rating, Next Rating Date, Previous Rating, Previous Rating Date, next_rating_date_or_end_of_data, credit_rating_year, previous_fixed_quarter_date, days_since_call]
Index: []


In [32]:
# Load ~\Box\STAT 222 Capstone\Intermediate Data\Calls\calls.csv
if DEBUG:
    calls = pd.read_parquet(r'~\Box\STAT 222 Capstone\Intermediate Data\Calls\combined_calls.parquet', nrows = 10)
else:
    calls = pd.read_parquet(r'~\Box\STAT 222 Capstone\Intermediate Data\Calls\combined_calls.parquet')
calls

,symbol,quarter,year,earnings_call_date,content,earnings_call_datetime,source,web
0,BCE,4,2012,2013-02-07,operators from ourselves. Our business perform...,2013-02-07,web,True
1,BCE,3,2012,2012-11-01,"Operator: Good morning, ladies and gentlemen. ...",2012-11-01,web,True
2,BCE,2,2012,2012-08-08,"Operator: Good morning, ladies and gentlemen. ...",2012-08-08,web,True
3,BCE,4,2013,2014-02-06,"Operator: Good morning, ladies and gentlemen. ...",2014-02-06,web,True
4,BCE,3,2013,2013-11-07,"Operator: Good morning, ladies and gentlemen. ...",2013-11-07,web,True
...,...,...,...,...,...,...,...,...
18457,WOR,1,2017,2016-09-28,"Operator: Good afternoon, ladies and gentlemen...",2016-09-28,web,True
18458,XIN,4,2017,2018-02-09,"Operator: Good day, everyone, and welcome to t...",2018-02-09,web,True
18459,XIN,3,2017,2017-11-11,"Operator: Good day, everyone, and welcome to t...",2017-11-11,web,True
18460,XIN,2,2017,2017-08-11,"Operator: Good day, everyone. And welcome to t...",2017-08-11,web,True


In [33]:
# check for duplicates on company by date
print(calls[calls.duplicated(subset=['symbol', 'earnings_call_date'], keep=False)])

Empty DataFrame
Columns: [symbol, quarter, year, earnings_call_date, content, earnings_call_datetime, source, web]
Index: []


# Earnings Call - Credit Score Merge

In [34]:
display(calls.columns)
display(cr_and_dates.columns)

Index(['symbol', 'quarter', 'year', 'earnings_call_date', 'content',
       'earnings_call_datetime', 'source', 'web'],
      dtype='object')

Index(['ticker', 'fixed_quarter_date', 'earnings_call_date', 'Rating',
       'Rating Agency Name', 'rating_date', 'Source', 'Rating Rank AAA is 10',
       'Next Rating', 'Next Rating Date', 'Previous Rating',
       'Previous Rating Date', 'next_rating_date_or_end_of_data',
       'credit_rating_year', 'previous_fixed_quarter_date', 'days_since_call'],
      dtype='object')

In [35]:
# convert earnings_call_date to date part
#cr_and_dates['earnings_call_date'] = pd.to_datetime(cr_and_dates['earnings_call_date']).dt.date
# drop earnings_call_datetime and web (identical as earnings_call_date and source)
calls.drop(["earnings_call_datetime", "web"], axis = 1, inplace=True)
# rename days_since_call and source 
cr_and_dates.rename(columns = {'days_since_call':'days_since_call_on_fixed_quarter', 'Source':'CR_source'}, inplace=True)
# rename symbol, source, and content
calls.rename(columns = {'symbol': 'ticker', "source": "Calls_source", "content": "transcript"}, inplace=True)

In [36]:
# print remaining columns
print(calls.columns)
print(calls.shape)
print(cr_and_dates.columns)
print(cr_and_dates.shape)
# Print type of columns
print(calls.dtypes)
print(cr_and_dates.dtypes)
# Print head of ticker, earnings_call_date in both datasets
print(calls[['ticker', 'earnings_call_date']].head())
print(cr_and_dates[['ticker', 'earnings_call_date']].head())
# Print all items with ticker == 'AAPL' in both datasets
print('aapl in calls')
print(calls[calls['ticker'] == 'AAPL'][['ticker', 'earnings_call_date']])
print('aapl in cr_and_dates')
print(cr_and_dates[cr_and_dates['ticker'] == 'AAPL'][['ticker', 'earnings_call_date']])

Index(['ticker', 'quarter', 'year', 'earnings_call_date', 'transcript',
       'Calls_source'],
      dtype='object')
(18462, 6)
Index(['ticker', 'fixed_quarter_date', 'earnings_call_date', 'Rating',
       'Rating Agency Name', 'rating_date', 'CR_source',
       'Rating Rank AAA is 10', 'Next Rating', 'Next Rating Date',
       'Previous Rating', 'Previous Rating Date',
       'next_rating_date_or_end_of_data', 'credit_rating_year',
       'previous_fixed_quarter_date', 'days_since_call_on_fixed_quarter'],
      dtype='object')
(8007, 16)
ticker                object
quarter                int64
year                   int64
earnings_call_date    object
transcript            object
Calls_source          object
dtype: object
ticker                              object
fixed_quarter_date                  object
earnings_call_date                  object
Rating                              object
Rating Agency Name                  object
rating_date                         object
CR_sourc

In [37]:
# Inner join cr_and_dates (on columns ticker, earnings_call_date) with calls (on columns company, date)
merged = pd.merge(cr_and_dates, calls, on=['ticker', 'earnings_call_date'], how='inner')
print(len(merged))

# Convert earnings_call_date to date format
merged['earnings_call_date'] = pd.to_datetime(merged['earnings_call_date']).dt.date

# Range of date, year, credit_rating_year
for variable in ['earnings_call_date', 'year', 'credit_rating_year']:
    print(variable)
    print(merged[variable].min())
    print(merged[variable].max())
    print()

# Free up memory from calls
del calls
del cr_and_dates
merged.head()

8007
earnings_call_date
2010-05-03
2016-09-30

year
2010
2017

credit_rating_year
2010
2016



,ticker,fixed_quarter_date,earnings_call_date,Rating,Rating Agency Name,rating_date,CR_source,Rating Rank AAA is 10,Next Rating,Next Rating Date,Previous Rating,Previous Rating Date,next_rating_date_or_end_of_data,credit_rating_year,previous_fixed_quarter_date,days_since_call_on_fixed_quarter,quarter,year,transcript,Calls_source
0,AAPL,2014-07-01 00:00:00.000000,2014-04-23,AA,Standard & Poor's Ratings Services,2014-05-27,Supplementary,9,AA,2015-02-18,AAA,2014-04-24,2015-02-18,2014,2014-04-01 00:00:00.000000,69,2,2014,"Operator: Good day, everyone, and welcome to t...",web
1,AAPL,2014-10-01 00:00:00.000000,2014-07-22,AA,Standard & Poor's Ratings Services,2014-05-27,Supplementary,9,AA,2015-02-18,AAA,2014-04-24,2015-02-18,2014,2014-07-01 00:00:00.000000,71,3,2014,"Operator: Good day, everyone, and welcome to t...",web
2,AAPL,2015-01-01 00:00:00.000000,2014-10-20,AA,Standard & Poor's Ratings Services,2014-05-27,Supplementary,9,AA,2015-02-18,AAA,2014-04-24,2015-02-18,2014,2014-10-01 00:00:00.000000,73,4,2014,"Operator: Good day, everyone, and welcome to t...",web
3,AAPL,2015-04-01 00:00:00.000000,2015-01-27,AA,Standard & Poor's Ratings Services,2015-02-18,Supplementary,9,AA,2015-05-28,AA,2014-05-27,2015-05-28,2015,2015-01-01 00:00:00.000000,64,1,2015,Operator: Good day ladies and gentlemen and we...,web
4,AAPL,2015-07-01 00:00:00.000000,2015-04-27,AA,Standard & Poor's Ratings Services,2015-06-02,Supplementary,9,AA,2015-08-25,AA,2015-05-28,2015-08-25,2015,2015-04-01 00:00:00.000000,65,2,2015,"Operator: Good day everyone, and welcome to th...",web


In [38]:
# Keep only the date 
merged["fixed_quarter_date"] = pd.to_datetime(merged["fixed_quarter_date"]).dt.date
merged["previous_fixed_quarter_date"] = pd.to_datetime(merged["previous_fixed_quarter_date"]).dt.date
merged["rating_date"] = pd.to_datetime(merged["rating_date"]).dt.date

In [39]:
# check for duplicates on ticker by fixed_quarter_date
print(merged[merged.duplicated(subset=['ticker', 'fixed_quarter_date'], keep=False)])

Empty DataFrame
Columns: [ticker, fixed_quarter_date, earnings_call_date, Rating, Rating Agency Name, rating_date, CR_source, Rating Rank AAA is 10, Next Rating, Next Rating Date, Previous Rating, Previous Rating Date, next_rating_date_or_end_of_data, credit_rating_year, previous_fixed_quarter_date, days_since_call_on_fixed_quarter, quarter, year, transcript, Calls_source]
Index: []


In [40]:
# check values of source
print(merged['Calls_source'].value_counts())

Calls_source
web    8007
Name: count, dtype: int64


In [41]:
print(merged.shape)
print(merged.columns)

(8007, 20)
Index(['ticker', 'fixed_quarter_date', 'earnings_call_date', 'Rating',
       'Rating Agency Name', 'rating_date', 'CR_source',
       'Rating Rank AAA is 10', 'Next Rating', 'Next Rating Date',
       'Previous Rating', 'Previous Rating Date',
       'next_rating_date_or_end_of_data', 'credit_rating_year',
       'previous_fixed_quarter_date', 'days_since_call_on_fixed_quarter',
       'quarter', 'year', 'transcript', 'Calls_source'],
      dtype='object')


In [42]:
print("number of fix_quarter_date = rating_date:", sum(merged.fixed_quarter_date == merged.rating_date))
print("number of fix_quarter_date after rating_date:", sum(merged.fixed_quarter_date < merged.rating_date))
print("number of earnings_call_date = rating_date:", sum(merged.earnings_call_date == merged.rating_date))
print("number of earnings_call_date after rating_date:", sum(merged.earnings_call_date < merged.rating_date))

number of fix_quarter_date = rating_date: 26
number of fix_quarter_date after rating_date: 0
number of earnings_call_date = rating_date: 47
number of earnings_call_date after rating_date: 1619


In [43]:
merged.days_since_call_on_fixed_quarter.describe()

count    8007.000000
mean       56.977395
std        14.826100
min         0.000000
25%        54.000000
50%        61.000000
75%        66.000000
max        91.000000
Name: days_since_call_on_fixed_quarter, dtype: float64

In [44]:
merged.rename(columns={"year": "calls_year"}, inplace=True)

In [45]:
merged[merged.calls_year != merged.credit_rating_year].head(10)

,ticker,fixed_quarter_date,earnings_call_date,Rating,Rating Agency Name,rating_date,CR_source,Rating Rank AAA is 10,Next Rating,Next Rating Date,Previous Rating,Previous Rating Date,next_rating_date_or_end_of_data,credit_rating_year,previous_fixed_quarter_date,days_since_call_on_fixed_quarter,quarter,calls_year,transcript,Calls_source
7,AAPL,2016-04-01,2016-01-26,AA,Standard & Poor's Ratings Services,2015-08-25,Supplementary,9,AA,2016-05-20,AA,2015-06-02,2016-05-20,2015,2016-01-01,66,1,2016,"Operator: Good day, ladies and gentlemen, and ...",web
13,ABB,2013-07-01,2013-04-24,A,Standard & Poor's Ratings Services,2012-05-17,Supplementary,8,NaN,NaN,BBB,2012-02-01,2016-12-31,2012,2013-04-01,68,1,2013,"Operator: Ladies and gentlemen, good morning o...",web
14,ABB,2013-10-01,2013-07-25,A,Standard & Poor's Ratings Services,2012-05-17,Supplementary,8,NaN,NaN,BBB,2012-02-01,2016-12-31,2012,2013-07-01,68,2,2013,"Operator: Ladies and gentlemen, good morning o...",web
15,ABB,2014-01-01,2013-10-24,A,Standard & Poor's Ratings Services,2012-05-17,Supplementary,8,NaN,NaN,BBB,2012-02-01,2016-12-31,2012,2013-10-01,69,3,2013,"Operator: Ladies and gentlemen, good morning o...",web
16,ABB,2014-04-01,2014-02-13,A,Standard & Poor's Ratings Services,2012-05-17,Supplementary,8,NaN,NaN,BBB,2012-02-01,2016-12-31,2012,2014-01-01,47,4,2013,operator and the maintenance leader wants he c...,web
17,ABB,2014-07-01,2014-04-29,A,Standard & Poor's Ratings Services,2012-05-17,Supplementary,8,NaN,NaN,BBB,2012-02-01,2016-12-31,2012,2014-04-01,63,1,2014,"Operator: Ladies and gentlemen, good morning, ...",web
18,ABB,2014-10-01,2014-07-23,A,Standard & Poor's Ratings Services,2012-05-17,Supplementary,8,NaN,NaN,BBB,2012-02-01,2016-12-31,2012,2014-07-01,70,2,2014,"Operator: Ladies and gentlemen, good morning o...",web
19,ABB,2015-01-01,2014-10-22,A,Standard & Poor's Ratings Services,2012-05-17,Supplementary,8,NaN,NaN,BBB,2012-02-01,2016-12-31,2012,2014-10-01,71,3,2014,"Operator: Ladies and gentlemen, good morning o...",web
20,ABB,2015-04-01,2015-02-05,A,Standard & Poor's Ratings Services,2012-05-17,Supplementary,8,NaN,NaN,BBB,2012-02-01,2016-12-31,2012,2015-01-01,55,4,2014,Operator: First question is from Mr. Ben Uglow...,web
21,ABB,2015-07-01,2015-04-29,A,Standard & Poor's Ratings Services,2012-05-17,Supplementary,8,NaN,NaN,BBB,2012-02-01,2016-12-31,2012,2015-04-01,63,1,2015,"Operator: Ladies and gentlemen, good morning o...",web


In [46]:
merged.columns

Index(['ticker', 'fixed_quarter_date', 'earnings_call_date', 'Rating',
       'Rating Agency Name', 'rating_date', 'CR_source',
       'Rating Rank AAA is 10', 'Next Rating', 'Next Rating Date',
       'Previous Rating', 'Previous Rating Date',
       'next_rating_date_or_end_of_data', 'credit_rating_year',
       'previous_fixed_quarter_date', 'days_since_call_on_fixed_quarter',
       'quarter', 'calls_year', 'transcript', 'Calls_source'],
      dtype='object')

### Merge Financial Statements Data to CR-Calls data

In [47]:
# Load tabular financial statement data
df = pd.read_parquet(r'~\Box\STAT 222 Capstone\Intermediate Data\Tabular_Fin\combined_corrected_tabular_financial_statements_data.parquet')

# Perform merge
merged = pd.merge(merged, df, left_on=['ticker', 'calls_year', 'quarter'], right_on=['symbol', 'calendarYear', 'period'], how='inner')    
display(merged.head())

,ticker,fixed_quarter_date,earnings_call_date,Rating,Rating Agency Name,rating_date,CR_source,Rating Rank AAA is 10,Next Rating,Next Rating Date,...,EBIT,common_plus_preferred_stock,workingCapital,Ratio_A,Ratio_B,Ratio_C,Ratio_D,Ratio_E,Altman_Z,fillingDate
0,AAPL,2014-07-01,2014-04-23,AA,Standard & Poor's Ratings Services,2014-05-27,Supplementary,9,AA,2015-02-18,...,13593000.0,21496000.0,27333000.0,0.065989,0.221594,8.770191,0.132692,0.480288,6.530889,2014-04-24
1,AAPL,2014-10-01,2014-07-22,AA,Standard & Poor's Ratings Services,2014-05-27,Supplementary,9,AA,2015-02-18,...,10282000.0,22139000.0,21744000.0,0.046207,0.168219,5.445584,0.097717,0.443623,4.324703,2014-07-23
2,AAPL,2015-01-01,2014-10-20,AA,Standard & Poor's Ratings Services,2014-05-27,Supplementary,9,AA,2015-02-18,...,11165000.0,23313000.0,5083000.0,0.048158,0.181691,8.770191,0.021925,0.375916,6.153503,2014-10-27
3,AAPL,2015-04-01,2015-01-27,AA,Standard & Poor's Ratings Services,2015-02-18,Supplementary,9,AA,2015-05-28,...,24246000.0,24187000.0,9792000.0,0.077155,0.284844,4.807177,0.037389,0.371059,3.985264,2015-01-28
4,AAPL,2015-07-01,2015-04-27,AA,Standard & Poor's Ratings Services,2015-06-02,Supplementary,9,AA,2015-08-25,...,18278000.0,25376000.0,9162000.0,0.069979,0.222095,8.770191,0.035077,0.386379,6.295942,2015-04-28


In [48]:
# Convert fillingDate to date format
merged['fillingDate'] = pd.to_datetime(merged['fillingDate']).dt.date

In [49]:
# check for duplicates on ticker by fixed_quarter_date
print(merged[merged.duplicated(subset=['ticker', 'fixed_quarter_date'], keep=False)])

Empty DataFrame
Columns: [ticker, fixed_quarter_date, earnings_call_date, Rating, Rating Agency Name, rating_date, CR_source, Rating Rank AAA is 10, Next Rating, Next Rating Date, Previous Rating, Previous Rating Date, next_rating_date_or_end_of_data, credit_rating_year, previous_fixed_quarter_date, days_since_call_on_fixed_quarter, quarter, calls_year, transcript, Calls_source, date, symbol, reportedCurrency, cik, acceptedDate, calendarYear, period, cashAndCashEquivalents, shortTermInvestments, cashAndShortTermInvestments, netReceivables, inventory, otherCurrentAssets, totalCurrentAssets, propertyPlantEquipmentNet, goodwill, intangibleAssets, goodwillAndIntangibleAssets, longTermInvestments, taxAssets, otherNonCurrentAssets, totalNonCurrentAssets, otherAssets, totalAssets, accountPayables, shortTermDebt, taxPayables, deferredRevenue, otherCurrentLiabilities, totalCurrentLiabilities, longTermDebt, deferredRevenueNonCurrent, deferredTaxLiabilitiesNonCurrent, otherNonCurrentLiabilities, 

### Drops based on date disagreements

In [50]:
# Drop if fillingDate is not in the fixed quarter
print('num cases where fillingDate not in fixed quarter', len(merged[(merged['fillingDate'] < merged['previous_fixed_quarter_date']) | (merged['fillingDate'] > merged['fixed_quarter_date'])]))
merged = merged[(merged['fillingDate'] >= merged['previous_fixed_quarter_date']) & (merged['fillingDate'] <= merged['fixed_quarter_date'])]
# Drop if fillingDate more than 45 days after earnings_call_date
print('num cases where fillingDate more than 45 days after earnings_call_date', len(merged[(merged['fillingDate'] - merged['earnings_call_date']) > pd.Timedelta('45 days 00:00:00')]))
merged = merged[(merged['fillingDate'] - merged['earnings_call_date']) <= pd.Timedelta('45 days 00:00:00')]

num cases where fillingDate not in fixed quarter 652
num cases where fillingDate more than 45 days after earnings_call_date 3


### New Columns: Rating Changes

In [51]:
# Variable for rating on previous fixed_quarter_date
# Sort by ticker, fixed_quarter_date
merged = merged.sort_values(by=['ticker', 'fixed_quarter_date'])
# Shift the rating by 1
merged['rating_on_previous_fixed_quarter_date'] = merged.groupby('ticker')['Rating'].shift(1)
merged[['ticker', 'fixed_quarter_date', 'Rating', 'rating_on_previous_fixed_quarter_date']].head(10)

,ticker,fixed_quarter_date,Rating,rating_on_previous_fixed_quarter_date
0,AAPL,2014-07-01,AA,NaN
1,AAPL,2014-10-01,AA,AA
2,AAPL,2015-01-01,AA,AA
3,AAPL,2015-04-01,AA,AA
4,AAPL,2015-07-01,AA,AA
5,AAPL,2015-10-01,AA,AA
6,AAPL,2016-01-01,AA,AA
7,AAPL,2016-04-01,AA,AA
8,AAPL,2016-07-01,AA,AA
9,AAPL,2016-10-01,AA,AA


In [52]:
# Rating change variables
# Get numeric values for Rating and rating_on_next_earnings_call_date
# Encode Ratings as ordinal
rating_order = ['AAA', 'AA', 'A', 'BBB', 'BB', 'B', 'CCC', 'CC', 'C', 'D']
# Reverse
rating_order = rating_order[::-1]
# Create dictionary mapping to number values
rating_dict = {rating: i for i, rating in enumerate(rating_order, 1)}
# Create new column "Rating Rank AAA is 10"
merged['Rating Rank AAA is 10'] = merged['Rating'].map(rating_dict)
# Flag for investment grade - Rating Rank >= 7.0
merged['Investment_Grade'] = merged['Rating Rank AAA is 10'] >= 7.0
# Create new column "rating_on_previous_fixed_quarter_date AAA is 10"
merged['rating_on_previous_fixed_quarter_date AAA is 10'] = merged['rating_on_previous_fixed_quarter_date'].map(rating_dict)

# Cross tab Rating and 'Rating Rank AAA is 10'
display(pd.crosstab(merged['Rating'], merged['Rating Rank AAA is 10']))
# Cross tab rating_on_previous_fixed_quarter_date and 'rating_on_previous_fixed_quarter_date AAA is 10'
display(pd.crosstab(merged['rating_on_previous_fixed_quarter_date'], merged['rating_on_previous_fixed_quarter_date AAA is 10']))

Rating Rank AAA is 10,1,2,3,4,5,6,7,8,9,10
Rating,,,,,,,,,,
A,0,0,0,0,0,0,0,1207,0,0
AA,0,0,0,0,0,0,0,0,255,0
AAA,0,0,0,0,0,0,0,0,0,128
B,0,0,0,0,1061,0,0,0,0,0
BB,0,0,0,0,0,1746,0,0,0,0
BBB,0,0,0,0,0,0,2061,0,0,0
C,0,16,0,0,0,0,0,0,0,0
CC,0,0,14,0,0,0,0,0,0,0
CCC,0,0,0,165,0,0,0,0,0,0


rating_on_previous_fixed_quarter_date AAA is 10,1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0,9.0,10.0
rating_on_previous_fixed_quarter_date,,,,,,,,,,
A,0,0,0,0,0,0,0,1127,0,0
AA,0,0,0,0,0,0,0,0,230,0
AAA,0,0,0,0,0,0,0,0,0,120
B,0,0,0,0,971,0,0,0,0,0
BB,0,0,0,0,0,1603,0,0,0,0
BBB,0,0,0,0,0,0,1918,0,0,0
C,0,15,0,0,0,0,0,0,0,0
CC,0,0,13,0,0,0,0,0,0,0
CCC,0,0,0,154,0,0,0,0,0,0


In [53]:
# Variable for change before next call
def change(row):
    if row['Rating Rank AAA is 10'] < row['rating_on_previous_fixed_quarter_date AAA is 10']:
        return 'Downgrade Since Last Fixed Quarter Date'
    elif row['Rating Rank AAA is 10'] > row['rating_on_previous_fixed_quarter_date AAA is 10']:
        return 'Upgrade Since Last Fixed Quarter Date'
    elif row['Rating Rank AAA is 10'] == row['rating_on_previous_fixed_quarter_date AAA is 10']:
        return 'Same As Last Fixed Quarter Date'
    else:
        return None

# Apply function to create new column
merged['Change Direction Since Last Fixed Quarter Date'] = merged.apply(change, axis = 1)

# Also numeric version 'Change Since Last Fixed Quarter Date'
merged['Change Since Last Fixed Quarter Date'] = merged['Rating Rank AAA is 10'] - merged['rating_on_previous_fixed_quarter_date AAA is 10']

# Cross tab 'Change Direction Since Last Fixed Quarter Date' and 'Change Since Last Fixed Quarter Date'
display(pd.crosstab(merged['Change Direction Since Last Fixed Quarter Date'], merged['Change Since Last Fixed Quarter Date']))

Change Since Last Fixed Quarter Date,-2.0,-1.0,0.0,1.0,2.0
Change Direction Since Last Fixed Quarter Date,,,,,
Downgrade Since Last Fixed Quarter Date,16,127,0,0,0
Same As Last Fixed Quarter Date,0,0,5834,0,0
Upgrade Since Last Fixed Quarter Date,0,0,0,164,18


### Sector and Other Textual Company Information

From Kaggle: https://www.kaggle.com/datasets/aramacus/usa-public-companies

Supplemented with manually looked up sector data for missing sectors

In [54]:
# Load CSV 'C:\Users\ijyli\Box\STAT 222 Capstone\Intermediate Data\Sectors\combined_sector_data.csv'
combined_sector_data = pd.read_csv(r'~\Box\STAT 222 Capstone\Intermediate Data\Sectors\combined_sector_data.csv')
combined_sector_data.head()

,Ticker,Description,Company Name,Sector,Industry Group,Industry,Sub-Industry,Comment
0,CTVA,"Corteva, Inc. is a global provider of seed and...",CORTEVA INC,Materials,Materials,Chemicals,Fertilizers & Agricultural Chemicals,"Producers of fertilizers, pesticides, potash o..."
1,ALCO,"Alico, Inc. is an agribusiness and land manage...",ALICO INC,Consumer Staples,"Food, Beverage & Tobacco",Food Products,Agricultural Products,Producers of agricultural products. Includes c...
2,LMNR,Limoneira Company is primarily an agribusiness...,LIMONEIRA CO,Consumer Staples,"Food, Beverage & Tobacco",Food Products,Agricultural Products,Producers of agricultural products. Includes c...
3,SANW,S&W Seed Company (S&W) is a multi-crop and mid...,S&W SEED CO,Consumer Staples,"Food, Beverage & Tobacco",Food Products,Agricultural Products,Producers of agricultural products. Includes c...
4,TRC,Tejon Ranch Co. is a diversified real estate d...,TEJON RANCH CO,Real Estate,Real Estate,Real Estate Management & Development,Diversified Real Estate Activities,Companies engaged in a diverse spectrum of rea...


In [55]:
combined_sector_data.shape

(3389, 8)

In [56]:
# Left join us_stocks with merged
merged = pd.merge(merged, combined_sector_data, left_on='ticker', right_on='Ticker', how='left')
merged

,ticker,fixed_quarter_date,earnings_call_date,Rating,Rating Agency Name,rating_date,CR_source,Rating Rank AAA is 10,Next Rating,Next Rating Date,...,Change Direction Since Last Fixed Quarter Date,Change Since Last Fixed Quarter Date,Ticker,Description,Company Name,Sector,Industry Group,Industry,Sub-Industry,Comment
0,AAPL,2014-07-01,2014-04-23,AA,Standard & Poor's Ratings Services,2014-05-27,Supplementary,9,AA,2015-02-18,...,None,NaN,AAPL,"Apple Inc. (Apple) designs, manufactures and m...",APPLE INC,Information Technology,Technology Hardware & Equipment,"Technology Hardware, Storage & Peripherals","Technology Hardware, Storage & Peripherals","Manufacturers of cellular phones, personal com..."
1,AAPL,2014-10-01,2014-07-22,AA,Standard & Poor's Ratings Services,2014-05-27,Supplementary,9,AA,2015-02-18,...,Same As Last Fixed Quarter Date,0.0,AAPL,"Apple Inc. (Apple) designs, manufactures and m...",APPLE INC,Information Technology,Technology Hardware & Equipment,"Technology Hardware, Storage & Peripherals","Technology Hardware, Storage & Peripherals","Manufacturers of cellular phones, personal com..."
2,AAPL,2015-01-01,2014-10-20,AA,Standard & Poor's Ratings Services,2014-05-27,Supplementary,9,AA,2015-02-18,...,Same As Last Fixed Quarter Date,0.0,AAPL,"Apple Inc. (Apple) designs, manufactures and m...",APPLE INC,Information Technology,Technology Hardware & Equipment,"Technology Hardware, Storage & Peripherals","Technology Hardware, Storage & Peripherals","Manufacturers of cellular phones, personal com..."
3,AAPL,2015-04-01,2015-01-27,AA,Standard & Poor's Ratings Services,2015-02-18,Supplementary,9,AA,2015-05-28,...,Same As Last Fixed Quarter Date,0.0,AAPL,"Apple Inc. (Apple) designs, manufactures and m...",APPLE INC,Information Technology,Technology Hardware & Equipment,"Technology Hardware, Storage & Peripherals","Technology Hardware, Storage & Peripherals","Manufacturers of cellular phones, personal com..."
4,AAPL,2015-07-01,2015-04-27,AA,Standard & Poor's Ratings Services,2015-06-02,Supplementary,9,AA,2015-08-25,...,Same As Last Fixed Quarter Date,0.0,AAPL,"Apple Inc. (Apple) designs, manufactures and m...",APPLE INC,Information Technology,Technology Hardware & Equipment,"Technology Hardware, Storage & Peripherals","Technology Hardware, Storage & Peripherals","Manufacturers of cellular phones, personal com..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6657,ZTS,2015-10-01,2015-08-04,BBB,Standard & Poor's Ratings Services,2015-01-30,Both,7,BBB,2015-11-03,...,Same As Last Fixed Quarter Date,0.0,ZTS,"Zoetis Inc. is focused on the discovery, devel...",ZOETIS INC,Health Care,"Pharmaceuticals, Biotechnology & Life Sciences",Pharmaceuticals,Pharmaceuticals,"Companies engaged in the research, development..."
6658,ZTS,2016-01-01,2015-11-03,BBB,Standard & Poor's Ratings Services,2015-11-03,Supplementary,7,BBB,2016-01-22,...,Same As Last Fixed Quarter Date,0.0,ZTS,"Zoetis Inc. is focused on the discovery, devel...",ZOETIS INC,Health Care,"Pharmaceuticals, Biotechnology & Life Sciences",Pharmaceuticals,Pharmaceuticals,"Companies engaged in the research, development..."
6659,ZTS,2016-04-01,2016-02-16,BBB,Standard & Poor's Ratings Services,2016-01-22,Both,7,BBB,2016-12-23,...,Same As Last Fixed Quarter Date,0.0,ZTS,"Zoetis Inc. is focused on the discovery, devel...",ZOETIS INC,Health Care,"Pharmaceuticals, Biotechnology & Life Sciences",Pharmaceuticals,Pharmaceuticals,"Companies engaged in the research, development..."
6660,ZTS,2016-07-01,2016-05-04,BBB,Standard & Poor's Ratings Services,2016-01-22,Both,7,BBB,2016-12-23,...,Same As Last Fixed Quarter Date,0.0,ZTS,"Zoetis Inc. is focused on the discovery, devel...",ZOETIS INC,Health Care,"Pharmaceuticals, Biotechnology & Life Sciences",Pharmaceuticals,Pharmaceuticals,"Companies engaged in the research, development..."


In [57]:
# check for duplicates on ticker by fixed_quarter_date
print(merged[merged.duplicated(subset=['ticker', 'fixed_quarter_date'], keep=False)])

Empty DataFrame
Columns: [ticker, fixed_quarter_date, earnings_call_date, Rating, Rating Agency Name, rating_date, CR_source, Rating Rank AAA is 10, Next Rating, Next Rating Date, Previous Rating, Previous Rating Date, next_rating_date_or_end_of_data, credit_rating_year, previous_fixed_quarter_date, days_since_call_on_fixed_quarter, quarter, calls_year, transcript, Calls_source, date, symbol, reportedCurrency, cik, acceptedDate, calendarYear, period, cashAndCashEquivalents, shortTermInvestments, cashAndShortTermInvestments, netReceivables, inventory, otherCurrentAssets, totalCurrentAssets, propertyPlantEquipmentNet, goodwill, intangibleAssets, goodwillAndIntangibleAssets, longTermInvestments, taxAssets, otherNonCurrentAssets, totalNonCurrentAssets, otherAssets, totalAssets, accountPayables, shortTermDebt, taxPayables, deferredRevenue, otherCurrentLiabilities, totalCurrentLiabilities, longTermDebt, deferredRevenueNonCurrent, deferredTaxLiabilitiesNonCurrent, otherNonCurrentLiabilities, 

In [58]:
merged.Sector.isnull().sum()

0

### Some Basic Stats

In [59]:
# Summarize columns
print(merged.info())
# print column names
for col in merged.columns:
    print(col)
print(merged.shape)
print(merged.ticker.nunique())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6662 entries, 0 to 6661
Columns: 165 entries, ticker to Comment
dtypes: bool(1), datetime64[ns](1), float64(114), int32(1), int64(9), object(39)
memory usage: 8.3+ MB
None
ticker
fixed_quarter_date
earnings_call_date
Rating
Rating Agency Name
rating_date
CR_source
Rating Rank AAA is 10
Next Rating
Next Rating Date
Previous Rating
Previous Rating Date
next_rating_date_or_end_of_data
credit_rating_year
previous_fixed_quarter_date
days_since_call_on_fixed_quarter
quarter
calls_year
transcript
Calls_source
date
symbol
reportedCurrency
cik
acceptedDate
calendarYear
period
cashAndCashEquivalents
shortTermInvestments
cashAndShortTermInvestments
netReceivables
inventory
otherCurrentAssets
totalCurrentAssets
propertyPlantEquipmentNet
goodwill
intangibleAssets
goodwillAndIntangibleAssets
longTermInvestments
taxAssets
otherNonCurrentAssets
totalNonCurrentAssets
otherAssets
totalAssets
accountPayables
shortTermDebt
taxPayables
deferredRevenue
other

In [60]:
print((merged.ticker == merged.Ticker).all())
merged.drop(columns=["Ticker"], inplace=True)
print(merged.shape)
print(merged.columns)

True
(6662, 164)
Index(['ticker', 'fixed_quarter_date', 'earnings_call_date', 'Rating',
       'Rating Agency Name', 'rating_date', 'CR_source',
       'Rating Rank AAA is 10', 'Next Rating', 'Next Rating Date',
       ...
       'rating_on_previous_fixed_quarter_date AAA is 10',
       'Change Direction Since Last Fixed Quarter Date',
       'Change Since Last Fixed Quarter Date', 'Description', 'Company Name',
       'Sector', 'Industry Group', 'Industry', 'Sub-Industry', 'Comment'],
      dtype='object', length=164)


## 80-20 Train Test Split

In [61]:
# Define share of data to be used for training
share_train = 0.8

In [62]:
# Set seed to 222
import random
random.seed(222)
# Create column train_test_80_20 with 80% probability of train
# Create list of random uniforms of length of merged
rand_uniforms = [random.uniform(0, 1) for _ in range(merged.shape[0])]
# Train and test based of of uniforms
merged['train_test_80_20'] = ['train' if x < share_train else 'test' for x in rand_uniforms]
# Print ticker and fixed_quarter_date 10 train items
print(merged[merged['train_test_80_20'] == 'train'][['ticker', 'fixed_quarter_date']].head(10))
# Print value counts
print(merged['train_test_80_20'].value_counts())

   ticker fixed_quarter_date
0    AAPL         2014-07-01
1    AAPL         2014-10-01
2    AAPL         2015-01-01
3    AAPL         2015-04-01
4    AAPL         2015-07-01
5    AAPL         2015-10-01
6    AAPL         2016-01-01
7    AAPL         2016-04-01
8    AAPL         2016-07-01
10    ABB         2013-04-01
train_test_80_20
train    5404
test     1258
Name: count, dtype: int64


## Exporting Data

In [63]:
# Save a parquet file as well
merged.to_parquet(r'~\Box\STAT 222 Capstone\Intermediate Data\All_Data\all_data_fixed_quarter_dates.parquet', index=False)

In [64]:
# Sample 100 rows to create all_data_sample.csv
if not DEBUG:
    all_data_sample = merged.sample(100)
    all_data_sample.to_csv(r'~\Box\STAT 222 Capstone\Intermediate Data\All_Data\all_data_fixed_quarter_dates_sample.csv', index=False)
    # also save to xlsx
    all_data_sample.to_excel(r'~\Box\STAT 222 Capstone\Intermediate Data\All_Data\all_data_fixed_quarter_dates_sample.xlsx', index=False)

## Fragmented Version to Store on GitHub

In [65]:
# Split dataset into pieces
num_pieces = 8
total_len_pieces = 0
# Delete previous pieces, all contents of '../../../Data/All_Data/All_Data_Fixed_Quarter_Dates'
import os
import shutil
folder = r'../../../Data/All_Data/All_Data_Fixed_Quarter_Dates'
for filename in os.listdir(folder):
    file_path = os.path.join(folder, filename)
    try:
        if os.path.isfile(file_path) or os.path.islink(file_path):
            os.unlink(file_path)
        elif os.path.isdir(file_path):
            shutil.rmtree(file_path)
    except Exception as e:
        print('Failed to delete %s. Reason: %s' % (file_path, e))
# Save pieces
if not DEBUG:
    for i in range(num_pieces):
        # start index for piece rows
        start_index = i * len(merged) // num_pieces
        # end index for piece rows
        end_index = (i + 1) * len(merged) // num_pieces
        # get piece
        piece = merged[start_index:end_index]
        piece.to_parquet(r'../../../Data/All_Data/All_Data_Fixed_Quarter_Dates/all_data_fixed_quarter_dates_piece_' + str(i) + '.parquet', index=False)
        display(piece.head(3))
        print(len(piece))
        total_len_pieces += len(piece)
    # save 100 row sample
    merged.sample(100).to_parquet(r'../../../Data/All_Data/All_Data_Fixed_Quarter_Dates_Sample/all_data_fixed_quarter_dates_sample.parquet', index=False)


,ticker,fixed_quarter_date,earnings_call_date,Rating,Rating Agency Name,rating_date,CR_source,Rating Rank AAA is 10,Next Rating,Next Rating Date,...,Change Direction Since Last Fixed Quarter Date,Change Since Last Fixed Quarter Date,Description,Company Name,Sector,Industry Group,Industry,Sub-Industry,Comment,train_test_80_20
0,AAPL,2014-07-01,2014-04-23,AA,Standard & Poor's Ratings Services,2014-05-27,Supplementary,9,AA,2015-02-18,...,None,NaN,"Apple Inc. (Apple) designs, manufactures and m...",APPLE INC,Information Technology,Technology Hardware & Equipment,"Technology Hardware, Storage & Peripherals","Technology Hardware, Storage & Peripherals","Manufacturers of cellular phones, personal com...",train
1,AAPL,2014-10-01,2014-07-22,AA,Standard & Poor's Ratings Services,2014-05-27,Supplementary,9,AA,2015-02-18,...,Same As Last Fixed Quarter Date,0.0,"Apple Inc. (Apple) designs, manufactures and m...",APPLE INC,Information Technology,Technology Hardware & Equipment,"Technology Hardware, Storage & Peripherals","Technology Hardware, Storage & Peripherals","Manufacturers of cellular phones, personal com...",train
2,AAPL,2015-01-01,2014-10-20,AA,Standard & Poor's Ratings Services,2014-05-27,Supplementary,9,AA,2015-02-18,...,Same As Last Fixed Quarter Date,0.0,"Apple Inc. (Apple) designs, manufactures and m...",APPLE INC,Information Technology,Technology Hardware & Equipment,"Technology Hardware, Storage & Peripherals","Technology Hardware, Storage & Peripherals","Manufacturers of cellular phones, personal com...",train


832


,ticker,fixed_quarter_date,earnings_call_date,Rating,Rating Agency Name,rating_date,CR_source,Rating Rank AAA is 10,Next Rating,Next Rating Date,...,Change Direction Since Last Fixed Quarter Date,Change Since Last Fixed Quarter Date,Description,Company Name,Sector,Industry Group,Industry,Sub-Industry,Comment,train_test_80_20
832,BZH,2014-10-01,2014-07-31,CCC,Standard & Poor's Ratings Services,2014-07-31,Both,4,CCC,2015-07-27,...,Same As Last Fixed Quarter Date,0.0,"Beazer Homes USA, Inc. is a geographically div...",BEAZER HOMES USA INC,Consumer Discretionary,Consumer Durables & Apparel,Household Durables,Homebuilding,Residential construction companies. Includes m...,train
833,BZH,2015-01-01,2014-11-12,CCC,Standard & Poor's Ratings Services,2014-07-31,Both,4,CCC,2015-07-27,...,Same As Last Fixed Quarter Date,0.0,"Beazer Homes USA, Inc. is a geographically div...",BEAZER HOMES USA INC,Consumer Discretionary,Consumer Durables & Apparel,Household Durables,Homebuilding,Residential construction companies. Includes m...,train
834,BZH,2015-04-01,2015-01-30,CCC,Standard & Poor's Ratings Services,2014-07-31,Both,4,CCC,2015-07-27,...,Same As Last Fixed Quarter Date,0.0,"Beazer Homes USA, Inc. is a geographically div...",BEAZER HOMES USA INC,Consumer Discretionary,Consumer Durables & Apparel,Household Durables,Homebuilding,Residential construction companies. Includes m...,train


833


,ticker,fixed_quarter_date,earnings_call_date,Rating,Rating Agency Name,rating_date,CR_source,Rating Rank AAA is 10,Next Rating,Next Rating Date,...,Change Direction Since Last Fixed Quarter Date,Change Since Last Fixed Quarter Date,Description,Company Name,Sector,Industry Group,Industry,Sub-Industry,Comment,train_test_80_20
1665,EAT,2014-07-01,2014-04-23,BBB,Standard & Poor's Ratings Services,2014-05-28,Supplementary,7,BBB,2016-08-11,...,Same As Last Fixed Quarter Date,0.0,"Brinker International, Inc. is a casual dining...",BRINKER INTL INC,Consumer Discretionary,Consumer Services,"Hotels, Restaurants & Leisure",Restaurants,"Owners and operators of restaurants, bars, pub...",train
1666,EAT,2014-10-01,2014-08-07,BBB,Standard & Poor's Ratings Services,2014-05-28,Supplementary,7,BBB,2016-08-11,...,Same As Last Fixed Quarter Date,0.0,"Brinker International, Inc. is a casual dining...",BRINKER INTL INC,Consumer Discretionary,Consumer Services,"Hotels, Restaurants & Leisure",Restaurants,"Owners and operators of restaurants, bars, pub...",train
1667,EAT,2015-01-01,2014-10-21,BBB,Standard & Poor's Ratings Services,2014-05-28,Supplementary,7,BBB,2016-08-11,...,Same As Last Fixed Quarter Date,0.0,"Brinker International, Inc. is a casual dining...",BRINKER INTL INC,Consumer Discretionary,Consumer Services,"Hotels, Restaurants & Leisure",Restaurants,"Owners and operators of restaurants, bars, pub...",train


...

833


,ticker,fixed_quarter_date,earnings_call_date,Rating,Rating Agency Name,rating_date,CR_source,Rating Rank AAA is 10,Next Rating,Next Rating Date,...,Change Direction Since Last Fixed Quarter Date,Change Since Last Fixed Quarter Date,Description,Company Name,Sector,Industry Group,Industry,Sub-Industry,Comment,train_test_80_20
4996,PPG,2012-04-01,2012-01-19,A,Standard & Poor's Ratings Services,2011-06-16,Supplementary,8,BBB,2012-06-29,...,Same As Last Fixed Quarter Date,0.0,"PPG Industries, Inc. manufactures and distribu...",PPG INDUSTRIES INC,Materials,Materials,Chemicals,Specialty Chemicals,Companies that primarily produce high value-ad...,train
4997,PPG,2012-07-01,2012-04-19,BBB,Standard & Poor's Ratings Services,2012-06-29,Supplementary,7,BBB,2012-07-31,...,Downgrade Since Last Fixed Quarter Date,-1.0,"PPG Industries, Inc. manufactures and distribu...",PPG INDUSTRIES INC,Materials,Materials,Chemicals,Specialty Chemicals,Companies that primarily produce high value-ad...,test
4998,PPG,2012-10-01,2012-07-19,BBB,Standard & Poor's Ratings Services,2012-07-31,Supplementary,7,A,2013-06-20,...,Same As Last Fixed Quarter Date,0.0,"PPG Industries, Inc. manufactures and distribu...",PPG INDUSTRIES INC,Materials,Materials,Chemicals,Specialty Chemicals,Companies that primarily produce high value-ad...,train


833


,ticker,fixed_quarter_date,earnings_call_date,Rating,Rating Agency Name,rating_date,CR_source,Rating Rank AAA is 10,Next Rating,Next Rating Date,...,Change Direction Since Last Fixed Quarter Date,Change Since Last Fixed Quarter Date,Description,Company Name,Sector,Industry Group,Industry,Sub-Industry,Comment,train_test_80_20
5829,TDS,2016-01-01,2015-10-30,BB,Standard & Poor's Ratings Services,2014-11-24,Supplementary,6,NaN,NaN,...,Same As Last Fixed Quarter Date,0.0,"Telephone and Data Systems, Inc. is a diversif...",TELEPHONE & DATA SYSTEMS INC,Communication Services,Telecommunication Services,Wireless Telecommunication Services,Wireless Telecommunication Services,Providers of primarily cellular or wireless te...,train
5830,TDS,2016-04-01,2016-02-19,BB,Standard & Poor's Ratings Services,2014-11-24,Supplementary,6,NaN,NaN,...,Same As Last Fixed Quarter Date,0.0,"Telephone and Data Systems, Inc. is a diversif...",TELEPHONE & DATA SYSTEMS INC,Communication Services,Telecommunication Services,Wireless Telecommunication Services,Wireless Telecommunication Services,Providers of primarily cellular or wireless te...,train
5831,TDS,2016-07-01,2016-05-06,BB,Standard & Poor's Ratings Services,2014-11-24,Supplementary,6,NaN,NaN,...,Same As Last Fixed Quarter Date,0.0,"Telephone and Data Systems, Inc. is a diversif...",TELEPHONE & DATA SYSTEMS INC,Communication Services,Telecommunication Services,Wireless Telecommunication Services,Wireless Telecommunication Services,Providers of primarily cellular or wireless te...,train


833


In [66]:
# check total piece length and length of merged
print(total_len_pieces)
print(len(merged))

6662
6662
